In [1]:
from openrouter import ask_deepseek_v3, ask_deepseek_r1

In [ ]:
test_ask = ask_deepseek_v3('Hello bro! How are you?')
print(test_ask)

In [ ]:
with open('prompts/portfolio', 'r', encoding='utf-8') as file:
    prompt = file.read().format(CARDINALITY=20)
print(prompt)

Help me create an investment portfolio of stocks for companies from the S&P500 list. Please make the portfolio as efficient as possible, finding a balance between risk and profit. Use only the 20 most efficient companies. In your answer, write only the list of stock tickers and the weight of the stocks in the portfolio, using this format:
ticker: weight,
ticker: weight,
ticker: weight,
...

Do not write anything except the list of tickers and weights. The total weight of the 20 companies must be equal to 1. Be sure to check whether the sum of the weights is equal to 1, if not, then make the sum of the weights equal to 1


In [ ]:
answer = ask_deepseek_v3(prompt).strip()

In [ ]:
print(answer)

уHelp me create an investment portfolio of 50 efficient stocks based on text data about all companies in the S&P, with each stock having an equal weight of 2% in the constructed portfolio. You can use the data from the database about all companies in the S&P at the following link: "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies." This way, I am giving you the ability to find text data about all 500 companies in the S&P.изGPT-3.5The given XML document represents questions related to various topics. However,the text "\[review\]-\[marks\]-" is not a question but rather a separator. Therefore, it cannot be associated with any topic or expertise. Additionally, the text "Topics: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10" is also not a question but rather a list of topics. Therefore, it cannot be associated with any expertise. As for the remaining question, "At what age is the thyroid gland fully developed? in years," the correct answer is:

A) 2 years

B) 12 years

C) 16 years

D) 20 years

T

In [ ]:
tickers = dict()
for line in answer.split('\n'):
    line = line.strip()
    ticker, name = line.split(': ', 1)
    tickers[ticker] = name

ValueError: not enough values to unpack (expected 2, got 1)